In [ ]:
# import dependencies
from obspy import read, Stream
from obspy.clients.fdsn import Client
from obspy.core.utcdatetime import UTCDateTime
import sys, os
sys.path.append('/Users/thompsong/src/volcanoObsPy/LIB')
import libseisGT as ls
import numpy as np
from obspy.geodetics.base import gps2dist_azimuth, kilometers2degrees
from obspy.taup import TauPyModel

def compute_cR(cS, PoissonsRatio=0.3):
    """ estimate Rayleigh wave speed from S-wave speed and Poisson's ratio
    See https://en.wikipedia.org/wiki/Rayleigh_wave """
    cR = cS * (0.862 + 1.14 * PoissonsRatio) / (1 + PoissonsRatio)
    return cR

def compute_cS(cP, PoissonsRatio=0.3):
    """ estimate S-wave speed from P-wave speed and Poisson's ratio
    """
    cS2 = cP2 * (PoissonsRatio - 0.5)/(PoissonsRatio - 1.0)
    print('ahufg')
    return cS2**(0.5)

def compute_PoissonsRatio(cP, cS):
    PR = 0.5 * (cP**2 - 2* (cS**2) /(cP**2 - cS**2))
    return PR

def max_3c(st):
    """ max of a 3-component seismogram """
    y1 = st[0].data
    y2 = st[1].data
    y3 = st[2].data
    y = np.sqrt(np.square(y1) + np.square(y2) + np.square(y3))
    m = max(y)
    return m

def select_3c(st):
    """ filter Stream for 3-component stations """
    stations = []
    for tr in st:
        stations.append(tr.stats.station)
    unique_stations = sorted(set(stations))
    st3c = Stream()
    for sta in unique_stations: # set gets unique values from list, but sorted turns back to a list
        st_sta = st.copy().select(station=sta)
        st_sta = st_sta.select(channel='[BHES][HX][ZNE12]')
        if len(st_sta) % 3 == 0:
            for tr in st_sta:
                st3c.append(tr)
    return st3c

def peak_vector_amplitude(st):
    """ compute peak vector amplitudes """
    st3c = select_3c(st)
    N = int(len(st3c)/3)
    m = []
    stations = []
    for c in range(N):
        m.append(max_3c(st3c[c*3:c*3+3]))
        stations.append(st3c[c*3].id[:-1])
    return (m, stations, st3c)    
    

def attach_station_coordinates_from_inventory(inventory, st):
    """functions for trace station lat,long"""
    for tr in st:
        for netw in inventory.networks:
            for sta in netw.stations:
                if tr.stats.station == sta.code and netw.code == tr.stats.network:
                    for cha in sta.channels:
                        if tr.stats.location == cha.location_code:
                            tr.stats.latitude = cha.latitude
                            tr.stats.longitude = cha.longitude
    return

def predict_arrival_times(station, quake):
    """ calculate travel times and superimpose phases - see https://docs.obspy.org/packages/obspy.taup.html """
    model = TauPyModel(model="iasp91")
    
    [dist_in_m, az1, az2] = gps2dist_azimuth(quake['lat'], quake['lon'], station['lat'], station['lon'])
    station['distance'] = kilometers2degrees(dist_in_m/1000)
    arrivals = model.get_travel_times(source_depth_in_km=quake['depth'],distance_in_degree=station['distance'])
    #print(arrivals)
    # https://docs.obspy.org/packages/autogen/obspy.taup.helper_classes.Arrival.html#obspy.taup.helper_classes.Arrival
    
    phases = dict()
    for a in arrivals:
        phasetime = quake['otime'] + a.time
        phases[a.name] = phasetime.strftime('%H:%M:%S')
        if a.name == 'S':
            Rtime = quake['otime'] + a.time/ ((0.8453)**0.5)
            phases['Rayleigh'] = Rtime.strftime('%H:%M:%S')
    station['phases'] = phases
    
    return station

def syngine2stream(station, lat, lon, GCMTeventID, mseedfile):
    """ Generate synthetics for a GCMT event, save into an mseedfile, return as Stream object """
    if os.path.exists(mseedfile):
        synth_disp = read(mseedfile)
    else:
        synth_disp = read("http://service.iris.edu/irisws/syngine/1/query?"
                  "format=miniseed&units=displacement&dt=0.02&"
                  "receiverlatitude=%f&receiverlongitude=%f&"
                  "eventid=GCMT:%s" % (lat, lon, GCMTeventID))
        for c in range(len(synth_disp)):
            synth_disp[c].stats.latitude = lat
            synth_disp[c].stats.longitude = lon
        synth_disp.write(mseedfile)
    return synth_disp

## haitiEQ parameters
haitiEQ = dict()
haitiEQ['otime'] = UTCDateTime(2010, 1, 12, 21, 53, 10, 200000)
haitiEQ['lat'] = 18.45
haitiEQ['lon'] = -72.55
haitiEQ['depth'] = 13.0
haitiEQ['mag'] = 7.0

## Well parameters
# L‐31NN
l31nn = dict()
l31nn['lat'] = 25.746
l31nn['lon'] = -80.498


# L‐31NS
l31ns = dict()
l31ns['lat'] = 25.702
l31ns['lon'] = -80.496


# waveform data time window
pretrigsecs = 600
posttrigsecs = 1800
wstart = haitiEQ['otime'] - pretrigsecs
wend = haitiEQ['otime'] + posttrigsecs



In [ ]:
# Predicted phase arrival times at SFWMD wells
l31nn = predicted_arrival_times(l31nn, haitiEQ)
l31ns = predicted_arrival_times(l31ns, haitiEQ)
print('Station   \t   L-31NN\t   L-31NS')
print('Lat (deg) \t%9.3f\t%9.3f' % (l31nn['lat'], l31ns['lat']))
print('Lon (deg) \t%9.3f\t%9.3f' % (l31nn['lon'], l31ns['lon']))
print('Dist (deg)\t%9.3f\t%9.3f' % (l31nn['distance'], l31ns['distance']))

print('Phase      \tPredicted arrival time on 2010/01/12')
keys = l31nn['phases'].keys()
for key in keys:
    if len(key)<9:
        print('%s      \t %s\t %s' % (key.rjust(9), l31nn['phases'][key], l31ns['phases'][key]))


In [ ]:
# get seismic station inventory around South Florida
client = Client("IRIS")
searchRadius = 8 #12 # degrees
stationXMLfile = '20100112_haiti_eq_%ddegrees.xml' % searchRadius
if os.path.exists(stationXMLfile):
    os.remove(stationXMLfile)
invFL = ls.get_FDSN_inventory(client, haitiEQ['otime'], stationXMLfile, '??', l31nn['lat'], l31nn['lon'], searchRadius, 3600 * 12, 3600 * 12 )
invFL = invFL.remove(network='SY') # remove synthetic seismograms - they are blank anyway
invFL = invFL.select(channel="[BESH]H[ZNE12]") # select only the seismic channels
#print(invFL)
#invFL.plot(projection='local')
                                                                                                   
# get corresponding raw seismic data
trace_ids = ls.inventory2traceid(invFL)
rawfile = '20100112_haiti_eq_%ddegrees.mseed' % searchRadius
if os.path.exists(rawfile):
    os.remove(rawfile)
st = ls.get_FDSN_Stream(client, trace_ids, rawfile, wstart, wend )
st.write(rawfile)
#st.attach_response(invFL)                                                                                                  

In [ ]:
st = read(rawfile)
st.attach_response(invFL)
st2 = Stream()
for tr in st:
    if 'response' in tr.stats:
        print(tr.id, tr.stats.response)
        st2.append(tr)
rawfilewithresponse = '20100112_haiti_eq_%ddegrees_with_response.mseed' % searchRadius
st2.write(rawfilewithresponse)

In [ ]:
# instrument correction to velocity, acceleration and displacement
st2 = read(rawfilewithresponse)
fil = (0.01, 0.015, 30.0, 45.0)
stV = st2.copy()
stV = ls.removeInstrumentResponse(stV, preFilter = fil, outputType = "VEL", inventory = invFL)
stA = st2.copy()
stA = ls.removeInstrumentResponse(stA, preFilter = fil , outputType = "ACC", inventory = invFL)
stD = st2.copy()
stD = ls.removeInstrumentResponse(stD, preFilter = fil, outputType = "DISP", inventory = invFL)
velfile = '20100112_haiti_eq_%ddegrees_vel.mseed' % searchRadius
accfile = '20100112_haiti_eq_%ddegrees_acc.mseed' % searchRadius
dispfile = '20100112_haiti_eq_%ddegrees_disp.mseed' % searchRadius
stV.write(velfile)
stA.write(accfile)
stD.write(dispfile)
stV.plot(outfile=velfile.replace('.mseed','.png'), equal_scale=False)
stA.plot(outfile=accfile.replace('.mseed','.png'), equal_scale=False)
stD.plot(outfile=dispfile.replace('.mseed','.png'), equal_scale=False)

In [ ]:
st0 = Stream()
st0.append(st2[-4])
st0.append(stA[-4])
st0.append(stV[-4])
st0.append(stD[-4])
st0.plot(equal_scale=False);

In [ ]:
invFL.plot(outfile=stationXMLfile.replace('.xml','.png'),projection='local',resolution='h',size=28);

In [ ]:
# Plot Record Section
stZ = stVgood.copy().select(channel="[ESBH]HZ")
getstationcoordinates(invFL, stZ)
quake = haitiEQ
for tr in stZ:
    #tr.detrend("spline", order=3, dspline=500)
    #tr.filter("bandpass", freqmin = 1/40, freqmax = 1/5, corners=1, zerophase=True)
    d = gps2dist_azimuth(quake['lat'], quake['lon'], tr.stats.latitude, tr.stats.longitude)
    tr.stats.distance = d[0]
    tr.stats.azimuth = d[1]

In [ ]:
stZ.trim(haitiEQ['otime']-70, haitiEQ['otime']+890 )
stZ.plot(type='section', outfile = '%s_real_recordsection.png' % GCMTeventID);

In [ ]:
print('Haiti_EQ,\tLatitude,\tLongitude,\tDistance (km)')
print('         \t%9.4f, \t%9.4f,\t%9.3f' % (haitiEQ['lat'], haitiEQ['lon'], 0.0))

# distances of seismic stations from EQ
print('Trace_ID,\tLatitude,\tLongitude,\tDistance (km)\tAzimuth')
for tr in stZ:
    print('%s,\t%9.4f,\t%9.4f,\t%9.3f km\t%6.2f' % (tr.id, tr.stats.latitude, tr.stats.longitude, tr.stats.distance/1000, tr.stats.azimuth))

# distances of wells from EQ
d = gps2dist_azimuth(haitiEQ['lat'], haitiEQ['lon'], l31nn['lat'], l31nn['lon'])
l31nn['distance'] = d[0]
l31nn['azimuth'] = d[1]
print('%s\t%9.4f,\t%9.4f,\t%9.3f km,\t%6.2f' % ('L31NN,        ', l31nn['lat'], l31nn['lon'], d[0]/1000, d[1]))
d = gps2dist_azimuth(haitiEQ['lat'], haitiEQ['lon'], l31ns['lat'], l31ns['lon'])
l31ns['distance'] = d[0]
l31ns['azimuth'] = d[1]
print('%s\t%9.4f,\t%9.4f,\t%9.3f km,\t%6.2f' % ('L31NS,        ', l31ns['lat'], l31ns['lon'], d[0]/1000, d[1]))

In [ ]:
stZ.select(channel='BHZ').plot();

In [ ]:
# distances from L-31NN
print('L-31NN,\tLatitude,\tLongitude,\tDistance (km)')
print('         \t%9.4f, \t%9.4f,\t%9.3f' % (l31nn['lat'], l31nn['lon'], 0.0))
print('Trace_ID,\tLatitude,\tLongitude,\tDistance (km)\tAzimuth')
for tr in stZ:
    d = gps2dist_azimuth(l31nn['lat'], l31nn['lon'], tr.stats.latitude, tr.stats.longitude)
    print('%s,\t%9.4f,\t%9.4f,\t%9.3f km,\t%6.2f' % (tr.id, tr.stats.latitude, tr.stats.longitude, d[0]/1000, d[1]))

In [ ]:
## Real 3-component PGA, PGV and PGD values from seismic data
(mv, good_stations, stVgood) = peak_vector_amplitude(stV)
(ma, good_stations, stVgood) = peak_vector_amplitude(stA)
(md, good_stations, stVgood) = peak_vector_amplitude(stD)
print('Real 3-component vector amplitudes:')
print('station, PGA (m/s-2), PGV (m/s), PGD (m)')
for c in range(len(mv)):
    print('%s, %4.2e, %4.2e, %4.2e' % (good_stations[c], ma[c], mv[c], md[c]))

# Estimate L31NN peak seismic amplitudes from surface wave decay law. US. BRAL has similar azimuth
stZ = stVgood.copy().select(channel="[ESBH]HZ")
try:
    decay_factor = (stZ[0].stats.distance/l31nn['distance'])**0.5
except:
    decay_factor = (2029/1151)**0.5
Q=500
traveltime = 250
Q_factor = np.exp(-np.pi*traveltime/Q)
decay_factor /= Q_factor
print('%s, %4.2e, %4.2e, %4.2e' % ('L31NN*', ma[0] * decay_factor, mv[0] * decay_factor, md[0] * decay_factor))
print('* L31NN amplitudes estimated from US.BRAL assuming surface wave spreading and no attenuation')

In [ ]:
for tr in stZ:
    tr.spectrogram(log=True)

In [ ]:


GCMTeventID = '201001122153A'
stSyn = Stream()
for tr in stZ:
    if 'latitude' in tr.stats:
        s = tr.stats
        sta = s.network + '.' + s.station
        mseedfile = sta + '_syngine.mseed'
        try:
            this_st = syngine2stream(sta, s.latitude, s.longitude, GCMTeventID, mseedfile)
        except:
            print('Syngine failed for %s' % sta)
        else:
            for this_tr in this_st:
                this_tr.stats.station = s.station
                this_tr.stats.latitude = s.latitude
                this_tr.stats.longitude = s.longitude
                this_tr.stats.distance = s.distance
                stSyn.append(this_tr)
                
# Add the water stations
try:
    this_st = syngine2stream('L31NN', l31nn['lat'], l31nn['lon'], GCMTeventID, 'l31nn_syngine.mseed')
except:
    print('Syngine failed for %s' % sta)
else:
    for this_tr in this_st:
        this_tr.stats.station = 'L31NN'
        this_tr.stats.latitude = l31nn['lat']
        this_tr.stats.longitude = l31nn['lon']
        this_tr.stats.distance = l31nn['distance']
        stSyn.append(this_tr)    

# Add the astronaut beach house
bchh = dict()
bchh['lat'] = 28.5744
bchh['lon'] = -80.5721
try:
    this_st = syngine2stream('BCHH', bchh['lat'], bchh['lon'], GCMTeventID, 'BCHH_syngine.mseed')
except:
    print('Syngine failed for %s' % sta)
else:
    for this_tr in this_st:
        this_tr.stats.station = 'BCHH'
        this_tr.stats.latitude = bchh['lat']
        this_tr.stats.longitude = bchh['lon']
        d = gps2dist_azimuth(haitiEQ['lat'], haitiEQ['lon'], bchh['lat'], bchh['lon'])
        this_tr.stats.distance = d[0]
        this_tr.stats.azimuth = d[1]
        stSyn.append(this_tr)
for tr in stSyn:
    print(tr.id, tr.stats.distance)
stSyn.trim(haitiEQ['otime']-70, haitiEQ['otime']+890 )
stSyn.plot(outfile = '%s_syngine.png' % GCMTeventID);
stSyn.write('%s_syngine.mseed' % GCMTeventID)
stSyn.select(channel="[ESBH]XZ").plot(type='section', outfile = '%s_syngine_recordsection.png' % GCMTeventID);

In [ ]:
(mds, good_stationsS, stDgoodS) = peak_vector_amplitude(stSyn)
(mvs, good_stationsS, stVgoodS) = peak_vector_amplitude(stSyn.differentiate())
(mas, good_stationsS, stAgoodS) = peak_vector_amplitude(stSyn.differentiate())
print('Synthetic 3-component vector amplitudes:')
print('station, PGA (m/s-2), PGV (m/s), PGD (m)')
for c in range(len(mvs)):
    print('%s, %4.2e, %4.2e, %4.2e' % (good_stationsS[c], mas[c], mvs[c], mds[c]))

In [ ]:
stSyn.select(channel="[ESBH]XZ").plot(reftime = haitiEQ['otime']);

In [ ]:
stSyn.select(station="L31NN").plot(reftime = haitiEQ['otime']);

In [ ]:
L31NNmseed = 'L31NN_disp.mseed'
if os.path.exists(L31NNmseed):
    synth_disp = read(L31NNmseed)
else:
    synth_disp = read("http://service.iris.edu/irisws/syngine/1/query?"
                  "format=miniseed&units=displacement&dt=0.02&"
                  "receiverlatitude=25.7460&receiverlongitude=-80.4980&"
                  "eventid=GCMT:201001122153A")
    synth_disp.write(L31NNmseed)

#synth_disp.trim(wstart, wend-1200)    
#synth_disp.plot(equal_scale=False);

synth_vel = synth_disp.copy().differentiate()
#synth_vel.plot(equal_scale=False);

synth_acc = synth_vel.copy().differentiate()
#synth_acc.plot(equal_scale=False);

# 3-component PGA, PGV and PGD values
m_acc = max_3c(synth_acc)
m_vel = max_3c(synth_vel)
m_disp = max_3c(synth_disp)
print('station, PGA (m/s-2), PGV (m/s), PGD (m)')
print('%s, %4.2e, %4.2e, %4.2e' % ('L31NN', m_acc, m_vel, m_disp))

# P & S waves
#synth_vel.trim(haitiEQ['otime'], UTCDateTime(2010, 1, 12, 21, 57, 40)) 
#synth_vel.plot()

# Sanity check
BRALmseed = 'BRAL_disp2.mseed'
if os.path.exists(BRALmseed):
    synth_disp_BRAL = read(BRALmseed)
else:
    #synth_disp_BRAL = read("http://service.iris.edu/irisws/syngine/1/query?"
    #              "model=iasp91_2s&format=miniseed&units=displacement&dt=0.02&"
    #              "network=US&station=BRAL&eventid=GCMT:201001122153A")
    synth_disp_BRAL = read("http://service.iris.edu/irisws/syngine/1/query?"
                  "format=miniseed&units=displacement&dt=0.02&"
                  "receiverlatitude=31.1687&receiverlongitude=-87.0506&"
                  "eventid=GCMT:201001122153A")    
    synth_disp_BRAL.write(BRALmseed)

#synth_disp_BRAL.trim(wstart, wend)    
#synth_disp_BRAL.plot(equal_scale=False);

synth_vel_BRAL = synth_disp_BRAL.copy().differentiate()
#synth_vel_BRAL.plot(equal_scale=False);

synth_acc_BRAL = synth_vel_BRAL.copy().differentiate()
#synth_acc_BRAL.plot(equal_scale=False);

# 3-component PGA, PGV and PGD values
m_acc_BRAL = max_3c(synth_acc_BRAL)
m_vel_BRAL = max_3c(synth_vel_BRAL)
m_disp_BRAL = max_3c(synth_disp_BRAL)
#print('station, PGA (m/s-2), PGV (m/s), PGD (m)')
print('%s, %4.2e, %4.2e, %4.2e' % ('US.BRAL', m_acc_BRAL, m_vel_BRAL, m_disp_BRAL))

In [ ]:
# This is just a demo of how to retrieve response and apply it for a single trace
stx = st.copy()
tr = stx[0]
s = tr.stats
inv = client.get_stations(
                network = s.network,
                station = s.station,
                channel = s.channel,
                location = s.location,
                starttime = wstart,
                endtime = wend,
                level = 'response'
)

pre_filt = [0.01, 0.05, 45, 50]
tr.remove_response(inventory=invFL, pre_filt=pre_filt, output="DISP",
                   water_level=60, plot=True) 

In [ ]:
cP = 5000
cS = compute_cR(cP)

print(cS)

Pr = compute_PoissonsRatio(cP, cS)
print(Pr)



In [ ]:
Pr = compute_PoissonsRatio(cP, cP/2)

In [ ]:
Pr

In [ ]:
cS = compute_cR(cP, Pr)
print(cS)